In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from classifier import Data

dataset = Data(
    data_path = "../data/imdb.train.sample.csv",
    polarities = {
      "negative": 0,
      "positive": 1
    },
    data_label='review',
    target_label='sentiment'
)

In [4]:
from classifier._neural import Encoder

enc = Encoder({
    'model': 'textattack/bert-base-uncased-imdb',
    'layers': [11]
})

In [5]:
enc.df_encode(dataset.data, col=dataset.data_label, label=dataset.data_path)

In [6]:
import numpy as np
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    return MDS(n_components=dim).fit_transform(data)

In [7]:
from typing import Generator
import torch
import numpy as np
import pandas as pd
from scipy.spatial import distance


def analysis(
        data: pd.DataFrame,
        dim: list = None
) -> Generator:
    if dim is None:
        dim = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]

    embed_col: np.ndarray = torch.stack(data[enc.col_name].tolist()).numpy()

    for d in dim:

        # create record to keep row data
        record: dict = {'dim': d}

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == enc.dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        # compute centroid means and dispersion for each cluster
        for label, group in data.groupby(dataset.target_label):
            record[f'centroid_{label}'] = np.mean(np.stack(group['reduced_embeds'].tolist(), axis=0), axis=0).tolist()
            record[f'dispersion_{label}'] = np.sum(distance.pdist(group['reduced_embeds'].tolist()))

        record['distance'] = distance.cdist([record['centroid_positive']], [record['centroid_negative']]).item(0)

        yield record

In [8]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.709004e+06,1.349562e+06,22.700799
1,576,1.709415e+06,1.349937e+06,22.701091
2,384,1.709337e+06,1.349875e+06,22.701175
3,192,1.709047e+06,1.349683e+06,22.701461
4,96,1.708582e+06,1.349409e+06,22.701736
5,48,1.707326e+06,1.348580e+06,22.702967
6,24,1.704465e+06,1.346718e+06,22.705300
7,12,1.697297e+06,1.341795e+06,22.712486
8,6,1.675842e+06,1.327215e+06,22.739022
9,3,1.587790e+06,1.272563e+06,22.910621


In [9]:
import pandas as pd

cluster_analysis = pd.DataFrame.from_records(list(analysis(dataset.data)))
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']]

,dim,dispersion_positive,dispersion_negative,distance
0,768,1.709004e+06,1.349562e+06,22.700799
1,576,1.709429e+06,1.349943e+06,22.701073
2,384,1.709368e+06,1.349906e+06,22.701080
3,192,1.709093e+06,1.349721e+06,22.701409
4,96,1.708548e+06,1.349361e+06,22.701763
5,48,1.707356e+06,1.348583e+06,22.702989
6,24,1.704511e+06,1.346745e+06,22.705339
7,12,1.697146e+06,1.341956e+06,22.713308
8,6,1.675997e+06,1.326900e+06,22.738131
9,3,1.580486e+06,1.271556e+06,22.918353


In [10]:
cluster_analysis[['dim', 'dispersion_positive', 'dispersion_negative', 'distance']].to_csv('./cluster_analysis.csv')